In [1]:
#import numpy as np
import pygame
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import random
import numpy as np
from tqdm import tqdm

pygame 2.1.2 (SDL 2.0.20, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
OCEAN_SIZE = 200 # either 40x40 or 200x200
FISH_REPRODUCTION_TIME = 3  # fish reproduces every 5 steps, A = 5
SHARK_INITIAL_ENERGY = 3 # E = 3  
SHARK_REPRODUCTION_TIME = 20 # B = 20

# Pygame constants # for 40x40 we will use cellsize 5
CELL_SIZE = 4  # Size of each cell, in pixels
WINDOW_SIZE = [OCEAN_SIZE*CELL_SIZE, OCEAN_SIZE*CELL_SIZE]  # Window size, in pixels, need chan

# Colors
BLUE = (0, 255, 0)   #  Fish
BLACK = (0, 0, 255)    #  Ocean
GRAY = (0,0,0) #  Shark

# Lets dreate the ocean
ocean = [[None for _ in range(OCEAN_SIZE)] for _ in range(OCEAN_SIZE)]

num_fish = 3000
num_sharks = 100

In [3]:
class Fish:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.reproduction_counter = 0
        self.reproduction_counterRandomized = random.randint(1,4)

    def get_empty_neighbours(self):
        adjust_cells = [(self.x-1, self.y), (self.x+1, self.y), (self.x, self.y-1), (self.x, self.y+1)]
        neighbours = [(nx % OCEAN_SIZE, ny % OCEAN_SIZE) for nx, ny in adjust_cells]
        empty_neighbours = [(nx, ny) for nx, ny in neighbours if ocean[nx][ny] is None]
        return empty_neighbours
    
    def move(self):
        empty_neighbours = self.get_empty_neighbours()

        if empty_neighbours:  # if there are empty neighbouring cells
            old_x, old_y = self.x, self.y
            new_x, new_y = random.choice(empty_neighbours)

            # Move to new position
            ocean[old_x][old_y] = None  # Clear old position
            self.x, self.y = new_x, new_y  # Update to new position
            ocean[new_x][new_y] = self  # Place fish in new position

            self.reproduction_counter += 1
            if self.reproduction_counter >= self.reproduction_counterRandomized:# FISH_REPRODUCTION_TIME:
                self.reproduce(old_x, old_y)  # Pass old position to the reproduce method  
                      
    def reproduce(self, old_x, old_y):
        ocean[old_x][old_y] = Fish(old_x, old_y)
        self.reproduction_counter = 0

In [4]:
class Shark:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.energy = SHARK_INITIAL_ENERGY
        self.reproduction_counter = 0

    def get_fish_neighbours(self): # same thing a empty but now we look for fishes
        adjust_cells = [(self.x-1, self.y), (self.x+1, self.y), (self.x, self.y-1), (self.x, self.y+1)]
        neighbours = [(nx % OCEAN_SIZE, ny % OCEAN_SIZE) for nx, ny in adjust_cells]
        fish_neighbours = [(nx, ny) for nx, ny in neighbours if isinstance(ocean[nx][ny], Fish)]
        return fish_neighbours

    def get_empty_neighbours(self):
        adjust_cells = [(self.x-1, self.y), (self.x+1, self.y), (self.x, self.y-1), (self.x, self.y+1)]
        neighbours = [(nx % OCEAN_SIZE, ny % OCEAN_SIZE) for nx, ny in adjust_cells]
        empty_neighbours = [(nx, ny) for nx, ny in neighbours if ocean[nx][ny] is None]
        return empty_neighbours

    def move(self):
        old_x, old_y = self.x, self.y  # For my idea of not deleting own spawns, we need to spawn them in old position
        
        fish_neighbours = self.get_fish_neighbours() # look for fish
        if fish_neighbours:  
            new_x, new_y = random.choice(fish_neighbours)
            self.eat(new_x, new_y)

        else:  # else if no fishes move to empty cell
            empty_neighbours = self.get_empty_neighbours()
            if empty_neighbours: 
                new_x, new_y = random.choice(empty_neighbours)
                ocean[old_x][old_y] = None      # 1 Clear old position
                self.x, self.y = new_x, new_y   # 2 Update to new position
                ocean[new_x][new_y] = self      # 3 Place shark in new position
                
        self.energy -= 1 #it will move so it losses energy

        if self.energy <= 0:  # if energy drops to 0 or below, shark dies
            ocean[self.x][self.y] = None
        else: 
            self.reproduction_counter += 1
            if self.reproduction_counter >= SHARK_REPRODUCTION_TIME:
                self.reproduce(old_x, old_y)

    def eat(self, new_x, new_y):
        if isinstance(ocean[new_x][new_y], Fish):  # 1 Check if the object is an instance of Fish
            ocean[new_x][new_y] = None      # 2 Remove the fish from the cell

        ocean[self.x][self.y] = None        # 3 Clear old position
        self.x, self.y = new_x, new_y       # 4 Update to new position
        ocean[new_x][new_y] = self          # 5 Place shark in new position
        self.energy = SHARK_INITIAL_ENERGY  # 6 Reset energy after eating

    def reproduce(self, old_x, old_y): # same as for fishes
        ocean[old_x][old_y] = Shark(old_x, old_y)
        self.reproduction_counter = 0

In [5]:

# Spawn fishes
for _ in range(num_fish):
    while True: 
        x = random.randint(0, OCEAN_SIZE-1)
        y = random.randint(0, OCEAN_SIZE-1)
        if ocean[x][y] is None: # so we sapwn all of them in different places
            ocean[x][y] = Fish(x, y)
            break

# Spawn sharks
for _ in range(num_sharks):
    while True:
        x = random.randint(0, OCEAN_SIZE-1)
        y = random.randint(0, OCEAN_SIZE-1)
        if ocean[x][y] is None:
            ocean[x][y] = Shark(x, y)
            break

In [6]:
fish_population  = []
shark_population = []

In [7]:
# Initialize Pygame
pygame.init()
screen = pygame.display.set_mode(WINDOW_SIZE)
clock = pygame.time.Clock()

frame_number = 0
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE):
            pygame.quit()
            continue
            #quit()

    for i in range(OCEAN_SIZE):
        for j in range(OCEAN_SIZE):
            if isinstance(ocean[i][j], Fish):
                ocean[i][j].move()
            elif isinstance(ocean[i][j],Shark):
                ocean[i][j].move()

    #lets record the populations over time
    fish_count = sum(isinstance(cell, Fish) for row in ocean for cell in row)
    shark_count = sum(isinstance(cell, Shark) for row in ocean for cell in row)
    fish_population.append(fish_count)
    shark_population.append(shark_count)
    
    screen.fill(BLACK)
    for i in range(OCEAN_SIZE):
        for j in range(OCEAN_SIZE):
            if isinstance(ocean[i][j], Fish):
                pygame.draw.rect(screen, BLUE, [j * CELL_SIZE, i * CELL_SIZE, CELL_SIZE, CELL_SIZE])
            elif isinstance(ocean[i][j], Shark):
                pygame.draw.rect(screen, GRAY, [j * CELL_SIZE, i * CELL_SIZE, CELL_SIZE, CELL_SIZE])

    pygame.display.flip()

    # Save the current frame to an image file.
    #pygame.image.save(screen, f"frame_{frame_number}.png")
    frame_number += 1

    clock.tick(60)


error: display Surface quit

In [14]:
from matplotlib.colors import ListedColormap, BoundaryNorm
import matplotlib.pyplot as plt
import os

def save_frames(num_frames):
    # Create a new directory to store the frames
    if not os.path.exists('frames'):
        os.makedirs('frames')

    # Define the color map and normalization
    colors = [(1, 1, 1), (0, 0, 1), (0.5, 0.5, 0.5)]  # white for empty, blue for fish, gray for shark
    cmap = ListedColormap(colors)
    boundaries = [-0.5, 0.5, 1.5, 2.5]  # boundaries for color map
    norm = BoundaryNorm(boundaries, cmap.N, clip=True)

    # Generate and save frames
    for i in tqdm(range(num_frames)):
        # Update the ocean
        for row in ocean:
            for cell in row:
                if isinstance(cell, Fish):
                    cell.move()
                elif isinstance(cell, Shark):
                    cell.move()

        # Convert ocean to a numerical 2D array for plotting
        ocean_array = [[0 if cell is None else (1 if isinstance(cell, Fish) else 2) for cell in row] for row in ocean]

        # Plot and save the frame
        plt.imshow(ocean_array, cmap=cmap, norm=norm)
        plt.savefig(f'frames/frame_{i}.png')
        plt.clf()  # Clear the current figure after saving the frame

    print("All frames saved.")



In [13]:
save_frames(100)

 14%|█▍        | 140/1000 [00:36<03:42,  3.86it/s]


KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>

In [ ]:
sharks2 = []
for shark in shark_population:
    sharks2.append(shark*100)

plt.figure()
plt.plot(fish_population, label = "Fish")
plt.plot( sharks2, label = "Sharks * 100")

#plt.plot(fish_population, shark_population)
plt.xlabel("generations/time")
plt.ylabel("Number of fishes/sharks")
plt.title("rescaled sharks to see fluctuations")
plt.legend()
#plt.savefig("SpecialCase.png")
plt.show()

In [ ]:
plt.figure()
plt.plot(fish_population, label = "Fish")
plt.plot( shark_population, label = "Sharks")
plt.xlabel("generations/time")
plt.ylabel("Number of fishes/sharks")
plt.legend()
#plt.savefig("SpecialCase.png")
plt.show()


In [ ]:
plt.figure()
plt.plot(fish_population, shark_population)
plt.xlabel("Fish")
plt.ylabel("Sharks")
plt.title("Equalibrium")
#plt.savefig("SpecialCase.png")
plt.show()

In [ ]:
''' Notes '''
'''
1. Add saving images not from pygame to track evolution of ocean.
2. Run the simulaiton for at least 1000 steps.
3. Rescale/zoomIn o equilibrium.
4. Show population varianc over time for sharks. I need to show how it changes over time.
5. Write a presentation showing how the syste works.
6. try to run code on new laptop. there may be errors due to refreshrate on MacBook.
7. maybe add lines to show how equilibium mathces with poplation graphs.
'''